In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Bases fornecidas
bases_originais = [
    "vgg19_avg_256.csv",
    "vgg16_max_256.csv",
    "vgg16_max_128.csv",
    "vgg16_avg_128.csv",
    "vgg19_max_256.csv",
    "vgg16_avg_256.csv"
]
bases_pca = [nome.replace(".csv", "_pca10.csv") for nome in bases_originais]
bases_final = bases_originais + bases_pca

avaliacoes = ["70/30 Split", "10-fold CV"]
nb_models = {
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB(),
    "ComplementNB": ComplementNB()
}

resultados_nb = {}

for nome_arquivo in bases_final:
    caminho_arquivo = f"../../data/{nome_arquivo}"

    if not os.path.exists(caminho_arquivo):
        print(f"Arquivo não encontrado: {caminho_arquivo}")
        continue

    df = pd.read_csv(caminho_arquivo)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]

    # GaussianNB usa dados padronizados (centrados)
    X_std = StandardScaler().fit_transform(X)
    # MultinomialNB e ComplementNB requerem dados positivos, então normalizamos MinMax para [0,1]
    X_pos = MinMaxScaler().fit_transform(X)

    for metodo in avaliacoes:
        chave = f"{nome_arquivo} - {metodo}"
        resultados_nb[chave] = {}
        for nome_modelo, modelo in nb_models.items():
            X_model = X_std if nome_modelo == "GaussianNB" else X_pos
            try:
                if metodo == "70/30 Split":
                    X_train, X_test, y_train, y_test = train_test_split(
                        X_model, y, test_size=0.3, stratify=y, random_state=42
                    )
                    acc = modelo.fit(X_train, y_train).score(X_test, y_test)
                else:  # 10-fold CV
                    acc = cross_val_score(modelo, X_model, y, cv=10, scoring='accuracy').mean()
                resultados_nb[chave][nome_modelo] = acc
            except Exception as e:
                print(f"Erro em {chave} - {nome_modelo}: {e}")
                resultados_nb[chave][nome_modelo] = np.nan

# Convertendo para DataFrame
df_nb = pd.DataFrame.from_dict(resultados_nb, orient='index')
df_nb.index.name = "Base - Avaliação"

# Salvando resultados
df_nb.to_csv("resultados_naive_bayes.csv")
df_nb.to_excel("resultados_naive_bayes.xlsx")

print("✅ Resultados Naive Bayes salvos em:")
print("- resultados_naive_bayes.csv / .xlsx")
